In [23]:
# from IPython import parallel
# rc = parallel.Client()
# dv = rc[:]
# rc.ids

[0, 1, 2, 3]

In [1]:
#%%px --local 
import os
import sys
sys.path.append(os.environ['HOME']+'/home/proj/stats/MCMC')
os.environ['PYTHONPATH'] = os.environ['HOME']+'/home/proj/stats/MCMC'

import csv

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

#%matplotlib inline
import model
import driver
import data

min Temp = 3.000000 K; max Temp = 100.000000 K
python version of blackbody


In [4]:
### read the file of fluxes and errors
### could make a numpy record array instead of separate ones for name, z, err, flux?
### nb. this version has the same length for each row

herus_file = "herus_phot.csv"
herus_file = "herus_phot_csv.csv"


speed_of_light = 299792.458 ### micron GHz

def toFloat(s):
    return np.float(s) if s!='' else np.nan

names = []
zarr = []
fluxes = []
errs = []
with open(herus_file, "rb") as csvfile:
    hreader = csv.reader(csvfile)
    headers = hreader.next()
    ncol = len(headers)
    name_column = headers=='Name'
    z_column = headers=='Redshift'
    flux_columns = [i for i in range(ncol) if headers[i].startswith('F')]
    err_columns = [i for i in range(ncol) if headers[i].startswith('E')]
    lambdas = speed_of_light/np.array([float(headers[i].split('_')[0][1:]) for i in flux_columns])  ### lambda in um
    lambdas_err = speed_of_light/np.array([float(headers[i].split('_')[0][1:]) for i in flux_columns])
    assert(np.all(lambdas==lambdas_err))   ### check correct order
    
    for row in hreader:
        names.append(row[name_column])
        zarr.append(row[z_column])
        fluxes.append([toFloat(row[c]) for c in flux_columns])
        errs.append([toFloat(row[c]) for c in err_columns])


In [32]:
### read the file of fluxes and errors
### could make a numpy record array instead of separate ones for name, z, err, flux?
### nb. this version deletes nonexistent entries, but puts err=np.nan if there is a flux but no entry
#### used in driver.py: readfluxes_DLC_2014
import re
Epat = r'^E\d+_'
Fpat = r'^F\d+_'
UL25 = True
fluxes = []
errs = []
names = []
zarr = []
with open(herus_file, "rb") as csvfile:
    hreader = csv.reader(csvfile)
    headers = np.array(hreader.next())
    ncol = len(headers)
    name_column = np.where(headers=='Name')[0][0]
    z_column = np.where(headers=='Redshift')[0][0]
    flux_columns = np.array([i for i in range(ncol) if re.match(Fpat, headers[i])])
    err_columns = np.array([i for i in range(ncol) if re.match(Epat, headers[i])])
    assert np.all(err_columns == flux_columns+1)

    lambda_obs = np.array([float(headers[i].split('_')[0][1:]) for i in flux_columns])  ### Freq in GHz
    lambda_err = np.array([float(headers[i].split('_')[0][1:]) for i in flux_columns])
    assert(np.all(lambda_obs==lambda_err))   ### check correct order
    
    for row in hreader:
        row = np.asarray(row)
        if np.all(row==''): continue  ## ignore blank lines
        z = np.float(row[z_column])
        dat = zip(row[flux_columns], row[err_columns], lambda_obs)
        dat_compressed = np.array([[np.float(f), toFloat(e), l] for f,e,l in dat if f!=''])
        #flux = np.array([np.float(c) for c in row[flux_columns] if c!=''])
        #err = np.array([np.float(c) for c in row[err_columns] if c!=''])
        flux = np.array(dat_compressed[:,0])
        sig = np.array(dat_compressed[:,1])
        lambda_obs_1 = np.array(dat_compressed[:,2])
        
        nu_obs = speed_of_light/lambda_obs_1
        if UL25:
            l25 = np.where(np.round(lambda_obs_1)==25)[0]
            if l25:
                sig[l25] = flux[l25]
                flux[l25] = 0

        nu_rest = nu_obs*(1+z)

        names.append(row[name_column])
        zarr.append(z)
        fluxes.append(flux)
        errs.append(sig)

elens = [len(e) for e in errs]


In [33]:
reload(data)
d = data.readfluxes_DLC_2014(filename=herus_file)
d1 = d[1]
print [di.name for di in d]
print d[-3].name, d[-3].d, d[-3].z
print d[-1].name, d[-1].d, d[-1].z

['IRAS00188-0856', 'IRAS00397-1312', 'IRAS01003-2238', 'IRAS03158+4227', 'IRAS03521+0028', 'IRAS05189-2524', 'IRAS06035-7102', 'IRAS06206-6315', 'IRAS07598+6508', 'IRAS08311-2459', 'IRAS08572+3915', 'IRAS09022-3615', 'IRAS10378+1109', 'IRAS10565+2448', 'IRAS11095-0238', 'IRAS12071-0444', 'IRAS13120-5453', 'IRAS13451+1232', 'IRAS14348-1447', 'IRAS14378-3651', 'IRAS15250+3609', 'IRAS15462-0450', 'IRAS16090-0139', 'IRAS17208-0014', 'IRAS19254-7245', 'IRAS19297-0406', 'IRAS20087-0308', 'IRAS20100-4156', 'IRAS20414-1651', 'IRAS20551-4250', 'IRAS22491-1808', 'IRAS23128-5919', 'IRAS23230-6926', 'IRAS23253-5415', 'IRAS23365+3604', 'Mrk1014', 'UGC5101', '3C273', 'Mrk231', 'Mrk273', 'Mrk463', 'Arp220-short', 'NGC6240', 'Arp220']
Arp220-short [  85.64651107   76.28        123.71         30.4137       12.0643
    4.1448      103.8         112.4           0.832         6.286     ] 0.018
Arp220 [  30.1     11.7      3.9      2.2      4.3      7.6     12.2     18.4
   24.4     32.2     39.7     50.9 

In [2]:
#%matplotlib osx
%run driver.py --format=DLC_2014 --fdir=./figs_DLC_2014_2 --odir=./out_DLC_2014_2 --file=./herus_phot.csv 0 1 2 3

/Volumes/Data/Users/jaffe/Library/Python/2.7/lib/python/site-packages/matplotlib-1.3.1-py2.7-macosx-10.6-intel.egg/matplotlib/__init__.py:1155: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


normal behaviour
filename: ./herus_phot.csv
fig dir: ./figs_DLC_2014_2/
out dir: ./out_DLC_2014_2/
data range None
DLC_UL = 1
format = 0
filetype = DLC_2014
which= [3, 2, 1, 0]
Using file ./herus_phot.csv
keywords: {'random': True, 'randomrestart': True, 'filetype': 'DLC_2014', 'format': 0, 'DLC_ul': True}
Two-Component beta = 2
Object[s] IRAS00188-0856(z=0.128)
done with chain 0. naccept=9368 (0.624492)

100% |########################################################################|
100% |########################################################################|



per block:  [2481 2501 1864    4 2501   22]
done with chain 1. naccept=207779 (0.692594)
per block:  [50000 50001 40800  2573 50001 14409]
chi2 at max = 23.959789
Max ln likelihood -11.979895 at parameters:
[ -4.06286082e+04   2.00000000e+00   5.07408935e+00   1.19166173e-01
   2.00000000e+00   3.16992381e+01]
            \log A_1 mean = -34964.4 +- 15704.2
                 T_1 mean = 17.1866 +- 8.25366
            \log A_2 mean = 0.121946 +- 0.0226082
                 T_2 mean = 31.6239 +- 0.682885
ln Pr of mean = -12.010634
chi2(mean) = 24.021268
Object[s] IRAS00397-1312(z=0.262)

100% |########################################################################|


...


done with chain 0. naccept=5069 (0.337911)

100% |########################################################################|
100% |########################################################################|



per block:  [  14 2501   24    7 2501   27]
done with chain 1. naccept=108362 (0.361205)
per block:  [ 3247 50001  2584  1321 50001  1213]
chi2 at max = 490.351610
Max ln likelihood -245.175805 at parameters:
[  1.29950146   2.          16.17873661   0.70169467   2.          36.95378181]
            \log A_1 mean = 1.14589 +- 0.0775266
                 T_1 mean = 21.0318 +- 2.62063
            \log A_2 mean = 0.531272 +- 0.0862176
                 T_2 mean = 39.0947 +- 1.08623
ln Pr of mean = -349.073463
chi2(mean) = 698.146926
One-Component
Object[s] IRAS00188-0856(z=0.128)


100% |########################################################################|


KeyboardInterrupt: 

In [2]:
#%%px --noblock
%run driver.py --format=DLC_2014 --fdir=./figs_DLC_2014_2 --odir=./out_DLC_2014_2 --file=./herus_phot.csv 0

/Volumes/Data/Users/jaffe/Library/Python/2.7/lib/python/site-packages/matplotlib-1.3.1-py2.7-macosx-10.6-intel.egg/matplotlib/__init__.py:1155: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


normal behaviour
filename: ./herus_phot.csv
fig dir: ./figs_DLC_2014_2/
out dir: ./out_DLC_2014_2/
data range None
DLC_UL = 1
format = 0
filetype = DLC_2014
which= [0]
Using file ./herus_phot.csv
keywords: {'random': True, 'randomrestart': True, 'filetype': 'DLC_2014', 'format': 0, 'DLC_ul': True}
Two-Component beta = 2
Object[s] IRAS00188-0856(z=0.128)
Number of prior resamples: 2
done with chain 0. naccept=5202 (0.346777)

100% |########################################################################|
100% |########################################################################|



per block:  [   8 2501   90   16 2501   91]
Number of prior resamples: 213
done with chain 1. naccept=200161 (0.667201)
per block:  [ 1468 50001  1070 49999 50001 47627]
chi2 at max = 23.960476
Max ln likelihood -11.980238 at parameters:
[   0.11958923    2.           31.6869833  -108.35336389    2.
   42.34247664]
            \log A_1 mean = 0.115103 +- 0.0432487
                 T_1 mean = 31.8705 +- 1.58634
            \log A_2 mean = -59.1526 +- 22.238
                 T_2 mean = 65.4563 +- 19.4743
ln Pr of mean = -12.250167
chi2(mean) = 24.500333
Object[s] IRAS00397-1312(z=0.262)

100% |########################################################################|


...


done with chain 0. naccept=14413 (0.960803)

100% |########################################################################|
100% |########################################################################|



per block:  [2501 2501 2194 2501 2501 2220]
done with chain 1. naccept=270536 (0.901784)
per block:  [50001 50001 36428 50001 50001 34109]
chi2 at max = 288100.292392
Max ln likelihood -144050.146196 at parameters:
[ -1.77075745e+04   2.00000000e+00   9.95222803e+00  -7.75763967e+04
   2.00000000e+00   7.09575007e+01]
            \log A_1 mean = -23758.9 +- 3584.59
                 T_1 mean = 34.9297 +- 22.7496
            \log A_2 mean = -65937.4 +- 10857.2
                 T_2 mean = 67.3392 +- 22.8166
ln Pr of mean = -144050.146196
chi2(mean) = 288100.292392
Object[s] IRAS22491-1808(z=0.078)


100% |########################################################################|


KeyboardInterrupt: 

In [ ]:
#%%px --noblock
%run driver.py --format=DLC_2014 --fdir=./figs_DLC_2014_2 --odir=./out_DLC_2014_2 --file=./herus_phot.csv 1

In [ ]:
#%%px --noblock
%run driver.py --format=DLC_2014 --fdir=./figs_DLC_2014_2 --odir=./out_DLC_2014_2 --file=./herus_phot.csv 2

In [ ]:
#%%px --noblock
%run driver.py --format=DLC_2014 --fdir=./figs_DLC_2014_2 --odir=./out_DLC_2014_2 --file=./herus_phot.csv 3

In [6]:
reload(driver)
ret123 = driver.writeTabAll([], fbase="DLC_2015", dirname="./DLC_2015/out_DLC_2015_1/", nodat=True, check=True)

./DLC_2015/out_DLC_2015_1/check0.npy
nobj, npar, ndat =  43 4 0
./DLC_2015/out_DLC_2015_1/check1.npy
nobj, npar, ndat =  43 3 0
./DLC_2015/out_DLC_2015_1/check2.npy
nobj, npar, ndat =  43 2 0
./DLC_2015/out_DLC_2015_1/check3.npy
nobj, npar, ndat =  43 6 0
./DLC_2015/out_DLC_2015_1/check4.npy
nobj, npar, ndat =  43 4 0
nn, npar, ndat, nt =  43 4 0 2
nn, npar, ndat, nt =  43 3 0 1
nn, npar, ndat, nt =  43 2 0 1
nn, npar, ndat, nt =  43 6 0 2
nn, npar, ndat, nt =  43 4 0 1
